In [1]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [39]:


# def gather_data(env):
#     num_trials = 10000
#     min_score = 50
#     sim_steps = 500
#     trainingX, trainingY = [], []

#     scores = []
#     for _ in range(num_trials):
#         observation = env.reset()
#         score = 0
#         training_sampleX, training_sampleY = [], []
#         for step in range(sim_steps):
#             # action corresponds to the previous observation so record before step
#             action = np.random.randint(0, 2)
#             one_hot_action = np.zeros(2)
#             one_hot_action[action] = 1
#             training_sampleX.append(observation)
#             training_sampleY.append(one_hot_action)

#             observation, reward, done, _ = env.step(action)
#             score += reward
#             if done:
#                 break
#         if score > min_score:
#             scores.append(score)
#             trainingX += training_sampleX
#             trainingY += training_sampleY

#     trainingX, trainingY = np.array(trainingX), np.array(trainingY)
#     print("Average: {}".format(np.mean(scores)))
#     print("Median: {}".format(np.median(scores)))
#     return trainingX, trainingY


def gather_data(env):
    num_trials = 10000
    min_score = -800
    sim_steps = 200
    trainingX, trainingY = [], []
    scores = []
    for _ in range (num_trials):
        observations = env.reset()
        score = 0
        training_sampleX, training_sampleY = [], []
        for _ in range(sim_steps):
            action = np.random.uniform(-2,2)
            one_hot_action = np.zeros(1)
            one_hot_action[0] = action
            training_sampleX.append(observations)
            training_sampleY.append(action)         
            observations, reward, done, _ = env.step(one_hot_action)
            if done:
                break
            score += reward
        if score > min_score:
            scores.append(score)
            trainingX += training_sampleX
            trainingY += training_sampleY
    trainingX, trainingY = np.array(trainingX), np.array(trainingY)
    return trainingX, trainingY

        

In [40]:
env = gym.make("Pendulum-v0")

In [41]:
trainingX, trainingY = gather_data(env)


In [5]:
trainingY.size/200

365.0

In [36]:
def create_model():
    model = Sequential()
    model.add(Dense(128, input_shape=(3,), activation="relu"))
    model.add(Dropout(0.6))

    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.6))

    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.6))

    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.6))

    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.6))
    model.add(Dense(1, activation="relu"))

    model.compile(loss="mean_squared_logarithmic_error",optimizer="adam",metrics=["accuracy"])
    return model

In [37]:
model = create_model()

In [38]:
model.fit(trainingX, trainingY, epochs=2)
scores = []
num_trials = 10
sim_steps = 200
for _ in range(num_trials):
    observation = env.reset()
    score = 0
    for _ in range(sim_steps):
        action = np.argmax(model.predict(observation.reshape(1,3)))
        observation, reward, done, _ = env.step([action,1])
        score += reward
        if done:            
            break
        print(score)
    scores.append(score)
print(np.mean(scores))

Train on 73000 samples
Epoch 1/2
73000/73000 [==============================] - 8s 114us/sample - loss: 0.1705 - accuracy: 0.0000e+00
Epoch 2/2
73000/73000 [==============================] - 7s 102us/sample - loss: 0.1527 - accuracy: 0.0000e+00
-6.574621205398349
-13.249217184927367
-20.160266417773244
-27.436783913866073
-35.19476740785969
-43.53161018037983
-52.52099037202164
-62.20893834760216
-72.51606577378705
-82.1441356695474
-91.10758024359775
-99.4460148546431
-107.22380044915349
-114.52804281466
-121.46524815062544
-128.15707470163494
-134.73563010379644
-141.33860156087374
-148.10428632146275
-155.16643786558615
-162.64884185573362
-170.65970184838264
-179.2861903897063
-188.58978368290062
-198.60309823042417
-208.6007472732467
-217.92282325556056
-226.59518322224028
-234.6688622190293
-242.21891762538075
-249.3417714568931
-256.15139412682873
-262.77480029799796
-269.34724790528304
-276.00732650890217
-282.8919184539313
-290.13092624406465
-297.84173675787906
-306.123620365

-116.10974087414543
-117.58251742360457
-118.75691248582625
-119.79186178736516
-120.83419984530966
-122.03210515472776
-123.54886033615277
-125.57615199178414
-128.345338306935
-132.13349125709638
-137.25896850553937
-144.06017445602365
-152.8538413795145
-163.8784909162806
-177.24291914765442
-190.60829322596553
-201.93206610890383
-211.2257179814268
-218.611618848901
-224.31020701911518
-228.60016983322416
-231.7740183211896
-234.1055011939611
-235.8336636758341
-237.16065911076137
-238.2582237103688
-239.27862977517114
-240.3676060352048
-241.6779937005969
-243.38343418990985
-245.69108417582316
-248.8511979698837
-253.1595268539837
-258.9465839424502
-266.5478958477077
-276.2545977848913
-288.255811545751
-302.5976163625962
-315.1119512589616
-325.5767835913452
-334.04920081965764
-340.6962863457686
-345.76852364616934
-349.55536528647184
-352.34445820947207
-354.3960353141582
-355.9333249983212
-357.1446025953294
-358.1919556148983
-359.2232785932584
-360.38560307963166
-361.8388

KeyboardInterrupt: 

In [32]:
env.close()
